In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import polar as pl
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim
from scipy.spatial.transform import Rotation as R

## 1. 📈 IMU and Gyro feature extraction

In [ ]:
class ImuExtractor(nn.Module):
    def __init__(self):
        super().__init__()

        k = 15  # todo try to change this
        self.lpf_acc = nn.Conv1d(3, 3, k, padding="same", groups=3, bias=False)
        self.lpf_gyro = nn.Conv1d(3, 3, k, padding="same", groups=3, bias=False)

    def forward(self, imu: torch.Tensor):
        B, C, T = imu.shape

        acc = imu[:, :3, :]  # acc_x, acc_y, acc_z
        gyro = imu[:, 3:6, :]  # gyro_x, gyro_y, gyro_z
        extra = imu[:, 6:, :]

        # 1) Magnitude
        acc_mag = torch.norm(acc, dim=1, keepdim=True) # todo try bias=True as author did
        gyro_mag = torch.norm(gyro, dim=1, keepdim=True)

        # 2) Jerk
        jerk = F.pad(acc_mag.diff(), (1, 0))
        gyro_delta = F.pad(gyro_mag.diff(), (1, 0))

        # 3) energy # todo try without this
        acc_pow = acc**2
        gyro_pow = gyro**2

        # LPF / HPF
        acc_lpf = self.lpf_acc(acc)
        acc_hpf = acc - acc_lpf
        gyro_lpf = self.lpf_gyro(gyro)
        gyro_hpf = gyro - gyro_lpf

        # fmt: off
        features = [
            acc, gyro,
            jerk, gyro_delta,
            acc_pow, gyro_pow,
            acc_lpf, acc_hpf, gyro_lpf, gyro_hpf,
        ]
        # fmt: on

        return torch.cat(features, dim=1)

dummy_inp = torch.randn((64, 6, 40))
ImuExtractor()(dummy_inp).shape

ValueError: Expected `quat` to have shape (4,) or (N, 4), got (64, 3, 40).

## 2. 🧠 Building Blocks

In [3]:
class SEBlock(nn.Module):
    def __init__(self, c_in, r):
        super().__init__()

        self.squeeze = nn.AdaptiveAvgPool1d(1)
        self.excitation = nn.Sequential(
            nn.Linear(c_in, c_in // r, bias=False),  # todo try with bias
            nn.ReLU(inplace=True),
            nn.Linear(c_in // r, c_in, bias=False),  # todo try with bias
            nn.Sigmoid(),
        )

    def forward(self, x: torch.Tensor):
        squeezed = self.squeeze(x).squeeze(-1)  # B x C
        weights = self.excitation(squeezed).unsqueeze(-1)  # B x C x 1

        return weights * x


SEBlock(6, 8)(dummy_inp).shape

/home/zeyadcode/.pyenv/versions/aienv/lib/python3.12/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


torch.Size([64, 6, 40])

## 🥟 Baseline Network

In [4]:
class BaseNet(nn.Module):
    def __init__(self, c_in, n_classes):
        super().__init__()

        # fmt: off
        self.cnn_layer = nn.Sequential(
            nn.Conv1d(c_in, 32, 3, padding="same"),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Dropout(0.1),
            
            nn.Conv1d(32, 64, 3, padding="same"),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.1),
        )
        
        self.pool_layer = nn.AdaptiveAvgPool1d(1)
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64, n_classes),
        )
        # fmt: on

    def forward(self, x: torch.Tensor):
        x = self.cnn_layer(x)
        x = self.pool_layer(x).squeeze(-1)
        x = self.fc_layer(x)
        return x


BaseNet(c_in=dummy_inp.shape[1], n_classes=5)(dummy_inp).shape

torch.Size([64, 5])

## 🧹 Data Cleaning

## 👷 Feature Engineering

In [5]:
def feature_engineer(df: pd.DataFrame, df_demographs: pd.DataFrame):
    df = pd.merge(
        dummy_df,
        dummy_demographs_df,
        how="left",
        on="subject",
    )
    
    

dummy_df = pd.read_csv("../data/raw/train.csv")
dummy_demographs_df = pd.read_csv("../data/raw/train_demographics.csv")
df = pd.merge(
    dummy_df,
    dummy_demographs_df,
    how="left",
    on="subject",
)

In [8]:
df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,-1.0,-1.0,71.0,1,30,0,1,186.0,55,30.0
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,-1.0,-1.0,-1.0,1,30,0,1,186.0,55,30.0
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,-1.0,-1.0,-1.0,1,30,0,1,186.0,55,30.0
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,-1.0,-1.0,-1.0,1,30,0,1,186.0,55,30.0
